### Goals of this project:

How much impact does being late or too spaced out at the first stop have downstream?

What is the impact of the layover at the start of the trip (the difference between the first top arrival and departure time)?

Does more layover lead to more stable headways (lower values for % headway deviation)?

How closely does lateness (ADHERENCE) correlate to headway?

What is the relationship between distance or time travelled since the start of a given trip and the headway deviation? Does headway become less stable the further along the route the bus has travelled?

How much of a factor does the driver have on headway and on-time performance? The driver is indicated by the OPERATOR variable.
How does direction of travel, route, or location affect the headway and on-time performance?

How does time of day or day of week affect headway and on-time performance? Can you detect an impact of school schedule on headway deviation (for certain routes and at certain times of day)?


Does weather have any effect on headway or on-time performance? To help answer this question, the file bna_2022.csv contains historical weather data recorded at Nashville International Airport.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
%matplotlib inline

In [48]:
pd.options.display.max_columns = None

In [4]:
headway = pd.read_csv('../data/Headway Data.csv')

In [5]:
headway.head()        

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,TIME_POINT_ABBR,ROUTE_STOP_SEQUENCE,TRIP_EDGE,LATITUDE,LONGITUDE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS
0,120211101,1,76447164,2021-11-01,7,1704,2088,297750,0,TO DOWNTOWN,HBHS,4.0,1,36.107575,-86.812719,14:10:00,13:59:21,14:12:00,-2.000000,14.0,15.983333,1.983333,0,0,1,0,0.0,0,2,12.650000
1,120211101,1,76447165,2021-11-01,7,1704,2088,297750,0,TO DOWNTOWN,21BK,3.0,0,36.138372,-86.800622,14:20:00,14:23:21,14:23:21,-3.350000,14.0,17.333333,3.333333,0,0,1,0,0.0,0,11,0.000000
2,120211101,1,76447166,2021-11-01,7,1704,2088,297750,0,TO DOWNTOWN,MCC5_9,2.0,2,36.167091,-86.781923,14:39:00,14:36:46,14:59:11,-20.183333,NaN,NaN,NaN,0,1,0,0,NaN,0,23,22.416666
3,120211101,1,76447167,2021-11-01,50,1704,2088,297749,0,TO DOWNTOWN,MLKS,7.0,1,36.161008,-86.800851,15:10:00,15:04:31,15:10:17,-0.283333,NaN,NaN,NaN,0,0,1,0,0.0,0,24,5.766666
4,120211101,1,76447168,2021-11-01,50,1704,2088,297749,0,TO DOWNTOWN,MCC5_11,5.0,2,36.167091,-86.781923,15:27:00,15:16:59,15:16:59,10.016666,NaN,NaN,NaN,0,0,1,0,NaN,0,25,0.000000


In [6]:
headway.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1981715 entries, 0 to 1981714
Data columns (total 30 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   CALENDAR_ID                int64  
 1   SERVICE_ABBR               int64  
 2   ADHERENCE_ID               int64  
 3   DATE                       object 
 4   ROUTE_ABBR                 int64  
 5   BLOCK_ABBR                 int64  
 6   OPERATOR                   int64  
 7   TRIP_ID                    int64  
 8   OVERLOAD_ID                int64  
 9   ROUTE_DIRECTION_NAME       object 
 10  TIME_POINT_ABBR            object 
 11  ROUTE_STOP_SEQUENCE        float64
 12  TRIP_EDGE                  int64  
 13  LATITUDE                   float64
 14  LONGITUDE                  float64
 15  SCHEDULED_TIME             object 
 16  ACTUAL_ARRIVAL_TIME        object 
 17  ACTUAL_DEPARTURE_TIME      object 
 18  ADHERENCE                  float64
 19  SCHEDULED_HDWY             float64
 20  AC

In [7]:
headway.isnull().sum()

CALENDAR_ID                       0
SERVICE_ABBR                      0
ADHERENCE_ID                      0
DATE                              0
ROUTE_ABBR                        0
BLOCK_ABBR                        0
OPERATOR                          0
TRIP_ID                           0
OVERLOAD_ID                       0
ROUTE_DIRECTION_NAME              0
TIME_POINT_ABBR                   0
ROUTE_STOP_SEQUENCE              41
TRIP_EDGE                         0
LATITUDE                          0
LONGITUDE                         0
SCHEDULED_TIME                    0
ACTUAL_ARRIVAL_TIME           54383
ACTUAL_DEPARTURE_TIME         54430
ADHERENCE                     54430
SCHEDULED_HDWY               432294
ACTUAL_HDWY                  473824
HDWY_DEV                     474368
ADJUSTED_EARLY_COUNT              0
ADJUSTED_LATE_COUNT               0
ADJUSTED_ONTIME_COUNT             0
STOP_CANCELLED                    0
PREV_SCHED_STOP_CANCELLED    402647
IS_RELIEF                   

In [8]:
len(headway['ROUTE_ABBR'].unique())

8

In [9]:
headway['ROUTE_ABBR'].unique()

array([ 7, 50, 22, 23,  3, 52, 55, 56], dtype=int64)

In [10]:
len(headway['OPERATOR'].unique())

458

In [11]:
bna_weather = pd.read_csv('../data/bna_weather.csv')

In [12]:
bna_weather.head()

,Date,key,class,expire_time_gmt,obs_id,obs_name,valid_time_gmt,day_ind,temp,wx_icon,icon_extd,wx_phrase,pressure_tend,pressure_desc,dewPt,heat_index,rh,pressure,vis,wc,wdir,wdir_cardinal,gust,wspd,max_temp,min_temp,precip_total,precip_hrly,snow_hrly,uv_desc,feels_like,uv_index,qualifier,qualifier_svrty,blunt_phrase,terse_phrase,clds,water_temp,primary_wave_period,primary_wave_height,primary_swell_period,primary_swell_height,primary_swell_direction,secondary_swell_period,secondary_swell_height,secondary_swell_direction
0,2021-11-01 00:53:00,KBNA,observation,1635753180,KBNA,Nashville,1635745980,N,51,26,2600,Cloudy,1.0,Rising,43,51,74,29.59,10.0,51.0,360.0,N,NaN,9.0,68.0,51.0,NaN,0.0,NaN,Low,51.0,0.0,NaN,NaN,NaN,NaN,OVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-11-01 01:30:00,KBNA,observation,1635755400,KBNA,Nashville,1635748200,N,50,29,2900,Partly Cloudy,NaN,NaN,42,50,74,29.59,10.0,50.0,350.0,N,NaN,8.0,NaN,NaN,NaN,0.0,NaN,Low,50.0,0.0,NaN,NaN,NaN,NaN,SCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-11-01 01:53:00,KBNA,observation,1635756780,KBNA,Nashville,1635749580,N,49,33,3300,Fair,NaN,NaN,42,49,77,29.59,10.0,47.0,350.0,N,NaN,6.0,NaN,NaN,NaN,0.0,NaN,Low,47.0,0.0,NaN,NaN,NaN,NaN,CLR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-11-01 02:53:00,KBNA,observation,1635760380,KBNA,Nashville,1635753180,N,48,33,3300,Fair,NaN,NaN,41,48,77,29.60,10.0,48.0,360.0,N,NaN,3.0,NaN,NaN,NaN,0.0,NaN,Low,48.0,0.0,NaN,NaN,NaN,NaN,CLR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-11-01 03:53:00,KBNA,observation,1635763980,KBNA,Nashville,1635756780,N,47,33,3300,Fair,1.0,Rising Rapidly,41,47,80,29.61,10.0,47.0,NaN,CALM,NaN,0.0,NaN,NaN,NaN,0.0,NaN,Low,47.0,0.0,NaN,NaN,NaN,NaN,CLR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#calculating headway deviation percentage - which is HDWY_DEV/SCHEDULED_HDWY
headway['Deviation_Percentage'] =headway['HDWY_DEV']/headway['SCHEDULED_HDWY']*100
headway

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,TIME_POINT_ABBR,ROUTE_STOP_SEQUENCE,TRIP_EDGE,LATITUDE,LONGITUDE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS,Deviation_Percentage
0,120211101,1,76447164,2021-11-01,7,1704,2088,297750,0,TO DOWNTOWN,HBHS,4.0,1,36.107575,-86.812719,14:10:00,13:59:21,14:12:00,-2.000000,14.0,15.983333,1.983333,0,0,1,0,0.0,0,2,12.650000,14.166664
1,120211101,1,76447165,2021-11-01,7,1704,2088,297750,0,TO DOWNTOWN,21BK,3.0,0,36.138372,-86.800622,14:20:00,14:23:21,14:23:21,-3.350000,14.0,17.333333,3.333333,0,0,1,0,0.0,0,11,0.000000,23.809521
2,120211101,1,76447166,2021-11-01,7,1704,2088,297750,0,TO DOWNTOWN,MCC5_9,2.0,2,36.167091,-86.781923,14:39:00,14:36:46,14:59:11,-20.183333,NaN,NaN,NaN,0,1,0,0,NaN,0,23,22.416666,NaN
3,120211101,1,76447167,2021-11-01,50,1704,2088,297749,0,TO DOWNTOWN,MLKS,7.0,1,36.161008,-86.800851,15:10:00,15:04:31,15:10:17,-0.283333,NaN,NaN,NaN,0,0,1,0,0.0,0,24,5.766666,NaN
4,120211101,1,76447168,2021-11-01,50,1704,2088,297749,0,TO DOWNTOWN,MCC5_11,5.0,2,36.167091,-86.781923,15:27:00,15:16:59,15:16:59,10.016666,NaN,NaN,NaN,0,0,1,0,NaN,0,25,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981710,120221101,1,91057724,2022-11-01,50,9302,2355,329980,0,TO DOWNTOWN,MCC4_20,3.0,2,36.167091,-86.781923,15:18:00,15:19:43,15:20:44,-2.733333,NaN,NaN,NaN,0,0,1,0,NaN,0,3,1.016666,NaN
1981711,120221101,1,91057851,2022-11-01,50,9950,1880,330003,0,TO DOWNTOWN,MLKS,10.0,1,36.161008,-86.800851,15:20:00,14:57:12,15:07:06,12.900000,10.0,7.633333,-2.366667,1,0,0,0,0.0,0,2,9.900000,-23.666670
1981712,120221101,1,91057852,2022-11-01,50,9950,1880,330003,0,TO DOWNTOWN,MCC5_11,4.0,2,36.167091,-86.781923,15:37:00,15:10:16,15:10:16,26.733333,NaN,NaN,NaN,1,0,0,0,NaN,0,3,0.000000,NaN
1981713,120221101,1,91057881,2022-11-01,56,9975,1922,330011,0,TO DOWNTOWN,MEIG,7.0,1,36.176017,-86.760399,15:20:00,14:59:40,15:19:02,0.966666,0.0,NaN,NaN,0,0,1,0,0.0,0,2,19.366666,NaN


In [14]:
#making a dataframe with only + headway deviations (HDWY_DEV)
headway1 = headway[['DATE', 'ROUTE_ABBR', 'BLOCK_ABBR', 'OPERATOR', 'TRIP_ID', 'ROUTE_DIRECTION_NAME', 'TRIP_EDGE', 'HDWY_DEV']]
headway1 = headway1.loc[(headway1['HDWY_DEV']>=0)]
headway1

,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,ROUTE_DIRECTION_NAME,TRIP_EDGE,HDWY_DEV
0,2021-11-01,7,1704,2088,297750,TO DOWNTOWN,1,1.983333
1,2021-11-01,7,1704,2088,297750,TO DOWNTOWN,0,3.333333
13,2021-11-01,22,2200,1352,298128,TO DOWNTOWN,0,0.866666
16,2021-11-01,22,2200,1352,298151,FROM DOWNTOWN,1,1.116666
17,2021-11-01,22,2200,1352,298151,FROM DOWNTOWN,0,0.400000
...,...,...,...,...,...,...,...,...
1981686,2022-11-01,7,705,1066,329447,FROM DOWNTOWN,1,2.000000
1981687,2022-11-01,7,705,1066,329447,FROM DOWNTOWN,0,2.233333
1981689,2022-11-01,7,706,2597,329460,TO DOWNTOWN,1,1.916666
1981690,2022-11-01,7,706,2597,329460,TO DOWNTOWN,0,1.650000


In [15]:
#making a dataframe with only - headway deviations (HDWY_DEV)
headway2 = headway[['DATE', 'ROUTE_ABBR', 'BLOCK_ABBR', 'OPERATOR', 'TRIP_ID', 'ROUTE_DIRECTION_NAME', 'TRIP_EDGE', 'HDWY_DEV']]
headway2 = headway2.loc[(headway2['HDWY_DEV']<0)]
headway2

,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,ROUTE_DIRECTION_NAME,TRIP_EDGE,HDWY_DEV
12,2021-11-01,22,2200,1352,298128,TO DOWNTOWN,1,-0.366667
14,2021-11-01,22,2200,1352,298128,TO DOWNTOWN,0,-1.016667
24,2021-11-01,22,2200,1352,298152,FROM DOWNTOWN,0,-3.666667
27,2021-11-01,22,2200,1352,298130,TO DOWNTOWN,1,-1.466667
34,2021-11-01,22,2200,1352,298131,TO DOWNTOWN,0,-0.083334
...,...,...,...,...,...,...,...,...
1981695,2022-11-01,7,706,375,329461,TO DOWNTOWN,1,-1.150000
1981696,2022-11-01,7,706,375,329461,TO DOWNTOWN,0,-2.500000
1981698,2022-11-01,7,706,375,329459,FROM DOWNTOWN,1,-1.883334
1981699,2022-11-01,7,706,375,329459,FROM DOWNTOWN,0,-6.516667


In [16]:

headway3 = headway[['DATE', 'ROUTE_ABBR', 'BLOCK_ABBR', 'OPERATOR', 'TRIP_ID', 'ROUTE_DIRECTION_NAME', 'TRIP_EDGE', 'HDWY_DEV']]
headway3 = headway3.loc[(headway3['TRIP_EDGE']==1)]
headway3

,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,ROUTE_DIRECTION_NAME,TRIP_EDGE,HDWY_DEV
0,2021-11-01,7,1704,2088,297750,TO DOWNTOWN,1,1.983333
3,2021-11-01,50,1704,2088,297749,TO DOWNTOWN,1,NaN
5,2021-11-01,22,2200,1352,298143,TO DOWNTOWN,1,NaN
9,2021-11-01,22,2200,1352,298162,FROM DOWNTOWN,1,NaN
12,2021-11-01,22,2200,1352,298128,TO DOWNTOWN,1,-0.366667
...,...,...,...,...,...,...,...,...
1981705,2022-11-01,50,8601,2600,329896,FROM DOWNTOWN,1,NaN
1981707,2022-11-01,55,8604,2374,329899,TO DOWNTOWN,1,NaN
1981709,2022-11-01,50,9302,2355,329980,TO DOWNTOWN,1,NaN
1981711,2022-11-01,50,9950,1880,330003,TO DOWNTOWN,1,-2.366667


In [17]:
#trip edge 2(the turn around)
headway4 = headway[['DATE', 'ROUTE_ABBR', 'BLOCK_ABBR', 'OPERATOR', 'TRIP_ID', 'ROUTE_DIRECTION_NAME', 'TRIP_EDGE', 'HDWY_DEV']]
headway4 = headway4.loc[(headway4['TRIP_EDGE']==2)]
headway4

,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,ROUTE_DIRECTION_NAME,TRIP_EDGE,HDWY_DEV
2,2021-11-01,7,1704,2088,297750,TO DOWNTOWN,2,NaN
4,2021-11-01,50,1704,2088,297749,TO DOWNTOWN,2,NaN
8,2021-11-01,22,2200,1352,298143,TO DOWNTOWN,2,NaN
11,2021-11-01,22,2200,1352,298162,FROM DOWNTOWN,2,NaN
15,2021-11-01,22,2200,1352,298128,TO DOWNTOWN,2,NaN
...,...,...,...,...,...,...,...,...
1981706,2022-11-01,50,8601,2600,329896,FROM DOWNTOWN,2,NaN
1981708,2022-11-01,55,8604,2374,329899,TO DOWNTOWN,2,NaN
1981710,2022-11-01,50,9302,2355,329980,TO DOWNTOWN,2,NaN
1981712,2022-11-01,50,9950,1880,330003,TO DOWNTOWN,2,NaN


In [18]:
#looking at hdwy_dev and adherence to scheduled time side by side 
headway5 = headway[['DATE', 'OPERATOR', 'TRIP_ID', 'TRIP_EDGE', 'ADHERENCE_ID', 'HDWY_DEV', 'ADHERENCE']]
headway5

,DATE,OPERATOR,TRIP_ID,TRIP_EDGE,ADHERENCE_ID,HDWY_DEV,ADHERENCE
0,2021-11-01,2088,297750,1,76447164,1.983333,-2.000000
1,2021-11-01,2088,297750,0,76447165,3.333333,-3.350000
2,2021-11-01,2088,297750,2,76447166,NaN,-20.183333
3,2021-11-01,2088,297749,1,76447167,NaN,-0.283333
4,2021-11-01,2088,297749,2,76447168,NaN,10.016666
...,...,...,...,...,...,...,...
1981710,2022-11-01,2355,329980,2,91057724,NaN,-2.733333
1981711,2022-11-01,1880,330003,1,91057851,-2.366667,12.900000
1981712,2022-11-01,1880,330003,2,91057852,NaN,26.733333
1981713,2022-11-01,1922,330011,1,91057881,NaN,0.966666


In [19]:
#time of day 

headway6 = headway[['DATE', 'OPERATOR', 'TRIP_ID', 'TRIP_EDGE', 'HDWY_DEV', 'ADHERENCE','SCHEDULED_TIME', 'ROUTE_ABBR']]
headway6 = headway6.loc[(headway6['TRIP_EDGE']==2) | headway6['TRIP_EDGE']==1] 
headway6 = headway6.loc[(headway6['OPERATOR']==2088)]
headway6

,DATE,OPERATOR,TRIP_ID,TRIP_EDGE,HDWY_DEV,ADHERENCE,SCHEDULED_TIME,ROUTE_ABBR
0,2021-11-01,2088,297750,1,1.983333,-2.000000,14:10:00,7
2,2021-11-01,2088,297750,2,NaN,-20.183333,14:39:00,7
3,2021-11-01,2088,297749,1,NaN,-0.283333,15:10:00,50
4,2021-11-01,2088,297749,2,NaN,10.016666,15:27:00,50
2984,2021-11-01,2088,300202,1,0.183333,-3.683333,05:53:00,52
...,...,...,...,...,...,...,...,...
1981358,2022-11-01,2088,329076,2,NaN,-4.333333,16:49:00,56
1981359,2022-11-01,2088,329081,1,1.766666,-1.916666,17:04:00,56
1981363,2022-11-01,2088,329081,2,NaN,-0.300000,17:45:00,56
1981364,2022-11-01,2088,329077,1,-2.383334,-1.166666,17:55:00,56


In [20]:
headway6['TIMES'] = headway6['SCHEDULED_TIME'].astype(str).str[:2].astype(int)
headway6

,DATE,OPERATOR,TRIP_ID,TRIP_EDGE,HDWY_DEV,ADHERENCE,SCHEDULED_TIME,ROUTE_ABBR,TIMES
0,2021-11-01,2088,297750,1,1.983333,-2.000000,14:10:00,7,14
2,2021-11-01,2088,297750,2,NaN,-20.183333,14:39:00,7,14
3,2021-11-01,2088,297749,1,NaN,-0.283333,15:10:00,50,15
4,2021-11-01,2088,297749,2,NaN,10.016666,15:27:00,50,15
2984,2021-11-01,2088,300202,1,0.183333,-3.683333,05:53:00,52,5
...,...,...,...,...,...,...,...,...,...
1981358,2022-11-01,2088,329076,2,NaN,-4.333333,16:49:00,56,16
1981359,2022-11-01,2088,329081,1,1.766666,-1.916666,17:04:00,56,17
1981363,2022-11-01,2088,329081,2,NaN,-0.300000,17:45:00,56,17
1981364,2022-11-01,2088,329077,1,-2.383334,-1.166666,17:55:00,56,17


In [34]:
headway6a = sorted(headway6['TIMES'].unique())
headway6a

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

In [35]:
len(headway6['TIMES'].unique())

20

In [36]:
len(headway6['DATE'].unique())

223

In [37]:
len(headway6['TRIP_ID'].unique())

344

In [39]:
len(headway6['ROUTE_ABBR'].unique())

7

In [40]:
#never drives on west end 3
#55 is galatin
#56 is murfresboro

headway6['ROUTE_ABBR'].unique()

array([ 7, 50, 52, 23, 22, 56, 55], dtype=int64)

In [41]:
headway['ROUTE_ABBR'].unique()

array([ 7, 50, 22, 23,  3, 52, 55, 56], dtype=int64)

In [54]:
headway7 = headway6[['DATE', 'OPERATOR', 'TRIP_ID', 'TRIP_EDGE', 'HDWY_DEV', 'ADHERENCE','SCHEDULED_TIME', 'ROUTE_ABBR']]
headway7 = headway7.loc[(headway7['TRIP_EDGE']==2) | headway7['TRIP_EDGE']==1] 
headway7 = headway7.loc[(headway7['OPERATOR']==2088)]
headway7 = headway7.loc[(headway7['ROUTE_ABBR']==7)]
headway7.head(25)

,DATE,OPERATOR,TRIP_ID,TRIP_EDGE,HDWY_DEV,ADHERENCE,SCHEDULED_TIME,ROUTE_ABBR
0,2021-11-01,2088,297750,1,1.983333,-2.000000,14:10:00,7
2,2021-11-01,2088,297750,2,NaN,-20.183333,14:39:00,7
5920,2021-11-02,2088,297750,1,-1.483334,-2.750000,14:10:00,7
5922,2021-11-02,2088,297750,2,NaN,-25.883333,14:39:00,7
23648,2021-11-05,2088,297750,1,-5.916667,-5.316666,14:10:00,7
23650,2021-11-05,2088,297750,2,NaN,-21.116666,14:39:00,7
35993,2021-11-08,2088,297750,1,2.016666,-2.083333,14:10:00,7
35995,2021-11-08,2088,297750,2,NaN,-14.750000,14:39:00,7
41904,2021-11-09,2088,297750,1,-0.583334,-1.916666,14:10:00,7
41906,2021-11-09,2088,297750,2,NaN,-30.366666,14:39:00,7


In [56]:
len(headway7['DATE'].unique())

40

In [57]:
len(headway7['TRIP_ID'].unique())

2

### Question 1: 
How much impact does being late or too spaced out at the first stop have downstream?